### test_dataset preprocessing  
1. 년도별 데이터셋 dict 형식으로 변경 dict[기업별][년도별][문장리스트]  
2. 뉴스 19년도부터의 데이터로 인해 잡플래닛 19년도 이하의 데이터들 제외  

In [1]:
import pandas as pd

In [2]:
df_2019 = pd.read_csv('./kosdaq_data/2019_kosdaq.csv', on_bad_lines='skip')
df_2020 = pd.read_csv('./kosdaq_data/2020_kosdaq.csv', on_bad_lines='skip')
df_2021 = pd.read_csv('./kosdaq_data/2021_kosdaq.csv', on_bad_lines='skip')
df_2022 = pd.read_csv('./kosdaq_data/2022_kosdaq.csv', on_bad_lines='skip')
df_2023 = pd.read_csv('./kosdaq_data/2023_kosdaq.csv', on_bad_lines='skip')
df_2024 = pd.read_csv('./kosdaq_data/2024_kosdaq.csv', on_bad_lines='skip')

df_news = pd.concat([df_2019,df_2020,df_2021,df_2022,df_2023,df_2024])

In [3]:
len(df_news)

594200

In [4]:
df_news

,company,content,date,title,url
0,오상헬스케어,기술력과 제품력을 갖춘 국내 중소벤처 바이오헬스케어 기업들이 세계 최대 가전IT 전...,2019.01.08,"재활훈련·건강검진… 기술中企, 글로벌 시장 노크",https://n.news.naver.com/mnews/article/029/000...
1,오상헬스케어,오상헬스케어036220의 현재 거래량이 최근 5거래일 거래량의 최고치를 넘어섰다&&...,2019.01.09,"[fnRASSI]오상헬스케어, 당일 거래량 증가하며 4거래일 하락세 지속",https://n.news.naver.com/mnews/article/014/000...
2,삼현,증시분석 전문기자 로봇 ET etbotetnews&&com&&전자신문과 금융AI전문...,2019.01.31,"[ET투자뉴스]삼현철강 18년4분기 실적 발표, 당기순이익 4.3억원… 전년 동기 ...",https://n.news.naver.com/mnews/article/030/000...
3,삼현,삼현철강은 1월 30일 11시 38분 전일 대비 약 4 상승한 5290원에 거래되고...,2019.01.30,"삼현철강, 전일 대비 약 4% 상승한 5,290원",https://n.news.naver.com/mnews/article/215/000...
4,삼현,전형진 기자 삼현철강은 보통주 1주당 150원의 현금배당을 실시하기로 했다고 ...,2019.01.31,"삼현철강, 주당 150원 현금배당키로",https://n.news.naver.com/mnews/article/015/000...
...,...,...,...,...,...
35171,리더스 기술투자,세종뉴시스손차민 기자 산업통상자원부가 올해 우수 중견기업 80곳에 1조원 규모의 ...,2024.04.15,"산업부, 올해 80개 중견기업에 1조 우대금융 지원",https://n.news.naver.com/mnews/article/003/001...
35172,리더스 기술투자,서울연합뉴스 김동규 기자 성장 잠재력이 높은 라이징 리더스로 선정된 중견기업에 올...,2024.04.15,'라이징 리더스' 중견기업 80곳에 1조원 규모 우대금융,https://n.news.naver.com/mnews/article/001/001...
35173,카스,이마트 이마트에브리데이 이마트24 3사가 맥주 수요가 높아지는 나들이 철을 맞아 대...,2024.04.17,"50개 맥주 최대 35% 할인...이마트 3사, '슈퍼비어 페스타'",https://n.news.naver.com/mnews/article/008/000...
35174,카스,오비맥주는 2024 대한민국 국제 맥주 대회KIBA 2024에서 12관왕을 달성했다...,2024.04.15,"오비맥주, ‘대한민국 국제 맥주 대회(KIBA 2024)’ 12관왕",https://n.news.naver.com/mnews/article/022/000...


In [5]:
# df_news = df_news.drop_duplicates(subset='content')
data = []


for _, row in df_news.iterrows():
    sentences = row['content'].split('&&')
    for sentence in sentences:
        if len(sentence) > 40:
            data.append((row['company'], row['date'], row['url'], sentence))
df_news = pd.DataFrame(data, columns=['Company','Date','URL', 'Sentence'])

In [11]:
df_news[:3]

,Company,Date,URL,Sentence
0,오상헬스케어,2019.01.08,https://n.news.naver.com/mnews/article/029/000...,기술력과 제품력을 갖춘 국내 중소벤처 바이오헬스케어 기업들이 세계 최대 가전IT 전...
1,오상헬스케어,2019.01.08,https://n.news.naver.com/mnews/article/029/000...,8일 바이오헬스케어 업계에 따르면 약 20개 국내 중소벤처 바이오헬스케어 기업들이 ...
2,오상헬스케어,2019.01.08,https://n.news.naver.com/mnews/article/029/000...,이번 전시회에서 잠재 고객을 확보하고 글로벌 시장 진출 기회를 잡겠다는 전략이다


In [12]:
df_news = df_news.drop_duplicates(subset='Sentence')

In [9]:
df_news['Sentence'][0]

'기술력과 제품력을 갖춘 국내 중소벤처 바이오헬스케어 기업들이 세계 최대 가전IT 전시회인 CES 2019에 대거 출동한다'

In [13]:
df_news['Date'] = df_news['Date'].apply(lambda x:x.split('.')[0])

C:\Users\SDA10\AppData\Local\Temp\ipykernel_20888\4136453136.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news['Date'] = df_news['Date'].apply(lambda x:x.split('.')[0])


In [14]:
df_news.head()

,Company,Date,URL,Sentence
0,오상헬스케어,2019,https://n.news.naver.com/mnews/article/029/000...,기술력과 제품력을 갖춘 국내 중소벤처 바이오헬스케어 기업들이 세계 최대 가전IT 전...
1,오상헬스케어,2019,https://n.news.naver.com/mnews/article/029/000...,8일 바이오헬스케어 업계에 따르면 약 20개 국내 중소벤처 바이오헬스케어 기업들이 ...
2,오상헬스케어,2019,https://n.news.naver.com/mnews/article/029/000...,이번 전시회에서 잠재 고객을 확보하고 글로벌 시장 진출 기회를 잡겠다는 전략이다
3,오상헬스케어,2019,https://n.news.naver.com/mnews/article/029/000...,AI를 접목한 홈 재활 서비스와 보조기기를 개발하는 네오펙트는 신제품 네오마노를 공개한다
4,오상헬스케어,2019,https://n.news.naver.com/mnews/article/029/000...,네오마노는 척수 손상 뇌졸중 루게릭 등 중추 신경 장애로 인해 손이 마비된 환자들이...


In [15]:
# '년도' 열을 숫자로 변환하고 변환할 수 없는 값은 NaN으로 설정
df_news['Date'] = pd.to_numeric(df_news['Date'], errors='coerce')

# NaN 값을 제거
df_news.dropna(subset=['Date'], inplace=True)

# '년도' 열을 정수형으로 변환
df_news['Date'] = df_news['Date'].astype(int)

C:\Users\SDA10\AppData\Local\Temp\ipykernel_20888\17262033.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news['Date'] = pd.to_numeric(df_news['Date'], errors='coerce')
C:\Users\SDA10\AppData\Local\Temp\ipykernel_20888\17262033.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news.dropna(subset=['Date'], inplace=True)
C:\Users\SDA10\AppData\Local\Temp\ipykernel_20888\17262033.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [16]:
len(df_news)

4571733

In [17]:
df_news.isnull().sum()

Company     0
Date        0
URL         0
Sentence    0
dtype: int64

In [28]:
job_df = pd.read_csv('./data/kosdaq_date_pros_cons_jobplanet_result.csv')

In [29]:
len(job_df)

56259

In [20]:
# '년도' 열을 숫자로 변환하고 변환할 수 없는 값은 NaN으로 설정
job_df['날짜'] = pd.to_numeric(job_df['날짜'], errors='coerce')

# NaN 값을 제거
job_df.dropna(subset=['날짜'], inplace=True)
job_df.dropna(inplace=True)
# '년도' 열을 정수형으로 변환
job_df['날짜'] = job_df['날짜'].astype(int)

In [26]:
len(job_df)

55480

In [25]:
job_df.isnull().sum()

Unnamed: 0    0
기업명           0
날짜            0
장점 리뷰         0
단점 리뷰         0
dtype: int64

In [27]:
# 기업별 날짜별 문장을 담을 딕셔너리
company_date_sentence = {}

# 뉴스
for _,token in df_news.iterrows():
    company_name = token['Company']
    sentences = token['Sentence']
    year = token['Date']

    if company_name not in company_date_sentence:
        company_date_sentence[company_name] = {year: [sentences]}
    else:
        if year not in company_date_sentence[company_name]:
            company_date_sentence[company_name][year] = [sentences]
        else:
            company_date_sentence[company_name][year].extend([sentences])

# 잡플래닛
for _, token in job_df.iterrows():
    company_name = token['기업명']
    pos_sentence = token['장점 리뷰']
    neg_sentence = token['단점 리뷰']
    year = token['날짜']
    if company_name not in company_date_sentence:
        company_date_sentence[company_name] = {year: [pos_sentence, neg_sentence]}
    else:
        if year not in company_date_sentence[company_name]:
            company_date_sentence[company_name][year] = [pos_sentence, neg_sentence]
        else:
            company_date_sentence[company_name][year].extend([pos_sentence, neg_sentence])

In [84]:
to_delete = []  # 삭제할 (회사, 연도) 쌍을 저장할 리스트

# 삭제할 항목 식별
for company, years in company_date_sentence.items():
    for year in range(2014, 2019):
        if year in years:
            to_delete.append((company, year))

# 식별된 항목 삭제
for company, year in to_delete:
    del company_date_sentence[company][year]

# 필요한 경우 회사 키도 삭제
empty_companies = [company for company, years in company_date_sentence.items() if not years]
for company in empty_companies:
    del company_date_sentence[company]

In [85]:
for i in company_date_sentence['삼현']:
    print(i)
    

2019
2020
2021
2022
2023
2024


In [87]:
import json

with open("total_kosdaq_result.json", "w") as f:
    json.dump(company_date_sentence, f)